The Australian company PSMA teamed up with DG to develop the product Geoscape: a diverse set of building attributes including height, rooftop material, solar panel installation and presence of a swimming pool in the property across the entire Australian continent. We used deep learning on GBDX to identify swimming pools in thousands of properties across Adelaide, a major city on the southern coast of Australia with a population of approximately one million. The full story is [here](http://gbdxstories.digitalglobe.com/swimming-pools/)

In [ ]:
# Specify your credentials and create a gbdx interface

import os
os.environ['GBDX_USERNAME'] = ''
os.environ['GBDX_PASSWORD'] = ''
os.environ['GBDX_CLIENT_ID'] = '' 
os.environ['GBDX_CLIENT_SECRET'] = ''

import gbdxtools
gbdx = gbdxtools.Interface()

In [ ]:
# Specify location of files needed in this story
input_location = 's3://gbd-customer-data/58600248-2927-4523-b44b-5fec3d278c09/platform-stories/swimming-pools'

In [ ]:
# Create the train task and set its input parameters

from os.path import join
train_task = gbdx.Task('train-cnn-classifier')
train_task.inputs.images = join(input_location, 'images')
train_task.inputs.geojson = join(input_location, 'train_geojson')
train_task.inputs.classes = 'No swimming pool, Swimming pool'     # Classes exactly as they appear in train.geojson

train_task.inputs.nb_epoch = '30'
train_task.inputs.nb_epoch_2 = '5'
train_task.inputs.train_size = '4500'
train_task.inputs.train_size_2 = '2500'
train_task.inputs.test_size = '1000'
train_task.inputs.bit_depth = '8'         # Provided imagery is dra'd

In [ ]:
# Create the deploy task and set its input parameters

deploy_task = gbdx.Task('deploy-cnn-classifier')
deploy_task.inputs.model = train_task.outputs.trained_model.value     # Trained model from train_task
deploy_task.inputs.images = join(input_location, 'images')
deploy_task.inputs.geojson = join(input_location, 'target_geojson')
deploy_task.inputs.classes = 'No swimming pool, Swimming pool'
deploy_task.inputs.bit_depth = '8'
deploy_task.inputs.min_side_dim = '10'    # Minimum acceptable side dimension for a polygon

In [ ]:
# String the tasks in a workflow
workflow = gbdx.Workflow([train_task, deploy_task])

# Set output location to platform-stories/trial-runs/random_str within your bucket/prefix
import uuid
random_str = str(uuid.uuid4())
output_location = join('platform-stories/trial-runs', random_str)

# save workflow outputs
workflow.savedata(train_task.outputs.trained_model, join(output_location, 'trained_model'))
workflow.savedata(deploy_task.outputs.classified_geojson, join(output_location, 'classified_geojson'))

In [ ]:
# Execute the workflow
workflow.execute()

In [ ]:
workflow.status

In [ ]:
# Check under this location for the output
print output_location

In [ ]:
# Create slippy map
from ipyleaflet import Map, TileLayer

m = Map(center=[-35.28, 138.46], zoom=15)

# This is IDAHO TMS url; we've saved the pansharpened strip in IDAHO format in a custom bucket and are serving it through the TMS service
url = 'http://idaho.geobigdata.io/v1/tile/platform-stories/d4060685-1ee2-492b-a5f9-fe68b376e13a/{z}/{x}/{y}?bands=0,1,2&token=' + gbdx.gbdx_connection.access_token

m.add_layer(TileLayer(url=url))
    
# launch map    
m

In [ ]:
# Download results
gbdx.s3.download(join(output_location, 'classified_geojson'))

In [ ]:
import json
with open('classified.geojson') as f:
    data = json.load(f)
        
n_properties = len(data['features'])
print 'There are ' + str(n_properties) + ' classified properties'

# Assign color based on classification
for feature in data['features']:
    if feature['properties']['CNN_class'] == 'Swimming pool':
        c = 'green'
    else:
        c = 'red'
    feature['properties']['style'] = {'color':c, 'fillOpacity':0}
    
from ipyleaflet import GeoJSON
g = GeoJSON(data=data)

# launch map
m.add_layer(g)
m